In [1]:
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import torch
import torch.nn as nn
from deepctr_torch.inputs import SparseFeat, DenseFeat, VarLenSparseFeat, get_feature_names
from deepctr_torch.models import *

import time
import os, copy
import numpy as np
import pandas as pd

2022-08-02 19:09:13.287047: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-02 19:09:13.290930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rh/devtoolset-8/root/usr/lib64:/opt/rh/devtoolset-8/root/usr/lib:/opt/rh/devtoolset-8/root/usr/lib64/dyninst:/opt/rh/devtoolset-8/root/usr/lib/dyninst:/opt/rh/devtoolset-8/root/usr/lib64:/opt/rh/devtoolset-8/root/usr/lib:/opt/rh/devtoolset-8/root/usr/lib64:/opt/rh/devtoolset-8/root/usr/lib:/opt/rh/devtoolset-8/root/usr/lib64/dyninst:/opt/rh/devtoolset-8/root/usr/lib/dyninst:/opt/rh/devtoolset-8/root/usr/lib64:/opt/rh/devtoolset-8/root/usr/lib
2022-0

# 特征处理

## 对变长特征进行编码处理

- 导入源域数据与目标域数据

In [142]:
target_train = pd.read_csv("ctr_data/train/train_data_ads.csv")
target_test = pd.read_csv('ctr_data/test/test_data_ads.csv')
target_all_data = pd.concat([target_train, target_test])

In [4]:
len(target_train), len(target_test), len(target_all_data)

(7675517, 976058, 8651575)

In [90]:
unique_table = target_all_data.nunique()

In [91]:
unique_table['user_id']

66882

In [31]:
# 统计所有特征的唯一数
target_all_data.nunique()

log_id                  1176633
label                         2
user_id                   66882
age                           8
gender                        3
residence                    36
city                        341
city_rank                     4
series_dev                   27
series_group                  7
emui_dev                     27
device_name                 258
device_size                1581
net_type                      6
task_id                   11984
adv_id                    13579
creat_type_cd                 9
adv_prim_id                 565
inter_type_cd                 4
slot_id                      61
site_id                       1
spread_app_id               116
hispace_app_tags             43
app_second_class             20
app_score                     3
ad_click_list_v001       119686
ad_click_list_v002       104914
ad_click_list_v003        66872
ad_close_list_v001         4095
ad_close_list_v002         2850
ad_close_list_v003         1827
pt_d    

- 对列表特征进行编码处理

In [5]:
target_varlen_sparse_feature_colnums = ['ad_click_list_v001', 
                                        'ad_click_list_v002', 
                                        'ad_click_list_v003', 
                                        'ad_close_list_v001', 
                                        'ad_close_list_v002',
                                        'ad_close_list_v003',
                                        'u_newsCatInterestsST']

In [6]:
target_sparse_feature_colnums = [
    'user_id',
    'age',
    'gender',
    'residence',
    'city',
    'city_rank',
    'series_dev',
    'series_group',
    'emui_dev',
    'device_name',
    'device_size',
    'net_type',
    'task_id',
    'adv_id',
    'creat_type_cd',
    'adv_prim_id',
    'inter_type_cd',
    'slot_id',
    'site_id',
    'spread_app_id',
    'hispace_app_tags',
    'app_second_class',
    'app_score',
    'pt_d',
    'u_refreshTimes',        # 信息流日均有效刷新次数
    'u_feedLifeCycle'        # 信息流用户活跃度
]

In [7]:
target_all_data

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0.0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
1,373253,1.0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
2,373252,1.0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
3,373251,0.0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
4,373255,0.0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030328,39^220^16,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976053,653084,NaN,287180,6,4,33,319,3,27,2,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206100920,65^192^71^199^20,7,17
976054,653074,NaN,287180,6,4,33,319,3,27,2,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206101155,65^192^71^199^20,7,17
976055,653076,NaN,287180,6,4,33,319,3,27,2,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206101157,65^192^71^199^20,7,17
976056,653077,NaN,287180,6,4,33,319,3,27,2,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206101158,65^192^71^199^20,7,17


In [8]:
# 稀疏特征编码
for feat in target_sparse_feature_colnums:
    lbe = LabelEncoder()
    target_all_data[feat] = lbe.fit_transform(target_all_data[feat])

In [42]:
target_all_data

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0.0,1,1,0,5,46,0,21,4,24,211,1116,5,1979,9318,4,453,2,35,0,25,36,3,2,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,532,39^220^16,0,5
1,373253,1.0,1,1,0,5,46,0,21,4,24,211,1116,5,894,11713,6,30,2,26,0,106,9,2,2,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,532,39^220^16,0,5
2,373252,1.0,1,1,0,5,46,0,21,4,24,211,1116,5,894,11713,6,30,2,26,0,106,9,2,2,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,532,39^220^16,0,5
3,373251,0.0,1,1,0,5,46,0,21,4,24,211,1116,5,8524,10162,6,296,2,36,0,22,38,6,2,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,532,39^220^16,0,5
4,373255,0.0,1,1,0,5,46,0,21,4,24,211,1116,5,10939,3607,6,17,2,33,0,90,32,7,2,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,534,39^220^16,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976053,653084,NaN,66881,4,2,22,218,1,16,0,0,44,1352,5,2100,7682,3,132,1,11,0,50,7,12,0,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,5926,65^192^71^199^20,7,7
976054,653074,NaN,66881,4,2,22,218,1,16,0,0,44,1270,5,4507,7279,8,196,2,5,0,110,28,4,2,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,6081,65^192^71^199^20,7,7
976055,653076,NaN,66881,4,2,22,218,1,16,0,0,44,1270,5,11095,1751,8,556,2,5,0,8,32,7,2,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,6083,65^192^71^199^20,7,7
976056,653077,NaN,66881,4,2,22,218,1,16,0,0,44,1270,5,8761,2554,8,17,2,5,0,90,32,7,2,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,6084,65^192^71^199^20,7,7


In [9]:
varlen_sparse_feature_values = target_all_data['ad_click_list_v001'].values

In [92]:
varlen_sparse_feature_values

array(['30157^30648^14278^31706', '30157^30648^14278^31706',
       '30157^30648^14278^31706', ..., '34890^21489^34426^22929^34057',
       '34890^21489^34426^22929^34057', '34890^21489^34426^22929^34057'],
      dtype=object)

In [10]:
varlen_sparse_feature_values[0].split('^')

['30157', '30648', '14278', '31706']

In [93]:
varlen_sparse_feature_values_lists = [values_list.split('^') for values_list in varlen_sparse_feature_values]

In [94]:
varlen_sparse_feature_values_lists

[['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227',

In [95]:
varlen_sparse_feature_values_dataframe = pd.DataFrame(data=varlen_sparse_feature_values_lists).replace([None], ['0'])

In [96]:
varlen_sparse_feature_values_dataframe

,0,1,2,3,4
0,30157,30648,14278,31706,0
1,30157,30648,14278,31706,0
2,30157,30648,14278,31706,0
3,30157,30648,14278,31706,0
4,30157,30648,14278,31706,0
...,...,...,...,...,...
8651570,34890,21489,34426,22929,34057
8651571,34890,21489,34426,22929,34057
8651572,34890,21489,34426,22929,34057
8651573,34890,21489,34426,22929,34057


In [15]:
len(varlen_sparse_feature_values_dataframe)

8651575

In [97]:
varlen_sparse_feature_flatten_values = varlen_sparse_feature_values_dataframe.values.flatten()

In [98]:
len(varlen_sparse_feature_values_dataframe.values.flatten())

43257875

In [99]:
varlen_sparse_feature_flatten_values

array(['30157', '30648', '14278', ..., '34426', '22929', '34057'],
      dtype=object)

In [101]:
lbe = LabelEncoder()
varlen_sparse_feature_encoder_values = lbe.fit_transform(varlen_sparse_feature_flatten_values)

In [19]:
len(varlen_sparse_feature_encoder_values)

43257875

In [102]:
lbe.classes_

array(['0', '10001', '10002', ..., '36351', '36352', '36353'],
      dtype=object)

In [30]:
len(lbe.classes_)

20922

In [20]:
len(varlen_sparse_feature_encoder_values[::5])

8651575

In [71]:
varlen_sparse_feature_values_dataframe[0]

0          30157
1          30157
2          30157
3          30157
4          30157
           ...  
8651570    34890
8651571    34890
8651572    34890
8651573    34890
8651574    34890
Name: 0, Length: 8651575, dtype: object

In [21]:
for i in range(5):
    varlen_sparse_feature_values_dataframe[i] = varlen_sparse_feature_encoder_values[i::5]

In [82]:
varlen_sparse_feature_values_dataframe

,0,1,2,3,4
0,15995,16378,3400,17232,0
1,15995,16378,3400,17232,0
2,15995,16378,3400,17232,0
3,15995,16378,3400,17232,0
4,15995,16378,3400,17232,0
...,...,...,...,...,...
8651570,19762,9123,19397,10254,19109
8651571,19762,9123,19397,10254,19109
8651572,19762,9123,19397,10254,19109
8651573,19762,9123,19397,10254,19109


In [22]:
dataframe_values = varlen_sparse_feature_values_dataframe.values
dataframe_values

array([[15995, 16378,  3400, 17232,     0],
       [15995, 16378,  3400, 17232,     0],
       [15995, 16378,  3400, 17232,     0],
       ...,
       [19762,  9123, 19397, 10254, 19109],
       [19762,  9123, 19397, 10254, 19109],
       [19762,  9123, 19397, 10254, 19109]])

- 更具dataframe编码，进获取embedding的掩码

In [24]:
varlen_sparse_feature_values_dataframe

,0,1,2,3,4
0,15995,16378,3400,17232,0
1,15995,16378,3400,17232,0
2,15995,16378,3400,17232,0
3,15995,16378,3400,17232,0
4,15995,16378,3400,17232,0
...,...,...,...,...,...
8651570,19762,9123,19397,10254,19109
8651571,19762,9123,19397,10254,19109
8651572,19762,9123,19397,10254,19109
8651573,19762,9123,19397,10254,19109


In [31]:
import torch.nn as nn

In [32]:
emd = nn.Embedding(len(lbe.classes_), embedding_dim=8)

In [33]:
emd

Embedding(20922, 8)

In [34]:
varlen_sparse_feature_values_dataframe.values.shape

(8651575, 5)

In [36]:
varlen_sparse_feature_values_embedding = emd(torch.from_numpy(varlen_sparse_feature_values_dataframe.values))

In [37]:
varlen_sparse_feature_values_embedding.shape

torch.Size([8651575, 5, 8])

- 掩码获取，再作用在编码上（因为存在0的部分）

In [113]:
datalist = target_all_data['ad_click_list_v001'].values

In [122]:
datalist = list(map(lambda x: [x.count('^') + 1], target_all_data['ad_click_list_v001'].values))

In [39]:
varlen_sparse_feature_values_lists

[['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['30157', '30648', '14278', '31706'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['28554', '22548', '13312', '29694', '11623'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227', '26741', '24226'],
 ['26227',

In [47]:
seq_length = list(map(lambda x: [len(x)], varlen_sparse_feature_values_lists))

In [49]:
len(seq_length), len(varlen_sparse_feature_values_lists)

(8651575, 8651575)

In [48]:
seq_length

[[4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5]

In [50]:
def _sequence_mask(lengths, maxlen=None, dtype=torch.bool):
    # Returns a mask tensor representing the first N positions of each cell.
    if maxlen is None:
        maxlen = lengths.max()
    row_vector = torch.arange(0, maxlen, 1).to(lengths.device)
    matrix = torch.unsqueeze(lengths, dim=-1)
    mask = row_vector < matrix

    mask.type(dtype)
    return mask

In [123]:
# 格式转换
seq_length = torch.tensor(seq_length)

/tmp/ipykernel_80646/3957356585.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_length = torch.tensor(seq_length)


In [124]:
mask = _sequence_mask(seq_length)

In [126]:
mask.shape

torch.Size([8651575, 1, 5])

In [67]:
mask.shape, varlen_sparse_feature_values_embedding.shape, seq_length.shape

(torch.Size([8651575, 1, 5]),
 torch.Size([8651575, 5, 8]),
 torch.Size([8651575, 1]))

In [130]:
emd.embedding_dim

8

In [73]:
# (nums, 1, colunms) -> (nums, colunms, 1) -> (nums, colunms, embedding_dims)
mask = torch.transpose(mask, 1, 2)

In [74]:
mask.shape

torch.Size([8651575, 5, 1])

In [75]:
embedding_size = varlen_sparse_feature_values_embedding.shape[-1]
mask = torch.repeat_interleave(mask, embedding_size, dim=2)

In [77]:
mask.shape

torch.Size([8651575, 5, 8])

In [78]:
hist = varlen_sparse_feature_values_embedding * mask.float()

## 数据处理

In [144]:
train = target_train.copy()

In [183]:
from utils.feature_deal import target_sparse_feature_names, target_varlen_sparse_feature_names, get_embedding_dict

In [160]:
embedding_dict, mask_dick, labelencoder_dick = \
        get_embedding_dict(target_all_data, target_sparse_feature_names, target_varlen_sparse_feature_names)

In [165]:
# 为列表数据构建子特征的dataframe
def get_varlen_sparse_feature_dataframe(dataframe, feature):
    varlen_sparse_feature_data = dataframe[feature].values        # 获取列表数据
    varlen_sparse_feature_datalists = [values_list.split('^') for values_list in varlen_sparse_feature_data]
    # 切分后的数据构建成dataframe形式，并用0来替换填充为None的数值
    varlen_sparse_feature_dataframe = pd.DataFrame(data=varlen_sparse_feature_datalists).replace([None], ['0'])
    return varlen_sparse_feature_dataframe

In [174]:
from collections import OrderedDict
# 获取训练集的编码表示用来后续切分训练
values_dick = OrderedDict()

for feat in target_sparse_feature_names:
    values_dick[feat] = labelencoder_dick[feat].fit_transform(target_all_data[feat])
for feat in target_varlen_sparse_feature_names:
    feat_dataframe = get_varlen_sparse_feature_dataframe(target_all_data, feat)
    feat_encoder_values = labelencoder_dick[feat].fit_transform(feat_dataframe.values.flatten())
#     for i in range(5):
#         feat_dataframe[i] = feat_encoder_values[i::5]
    values_dick[feat] = np.array(feat_encoder_values).reshape(-1, 5)

In [201]:
# from utils.feature_deal import target_sparse_feature_names, target_varlen_sparse_feature_names

x = [values_dick[feat] for feat in target_sparse_feature_names + target_varlen_sparse_feature_names]
for i in range(len(x)):
    if len(x[i].shape) == 1:X
        x[i] = np.expand_dims(x[i], axis=1)

In [208]:
X = np.concatenate(x, axis=-1)

In [209]:
X

array([[    1,     1,     0, ...,    62,     0,     0],
       [    1,     1,     0, ...,    62,     0,     0],
       [    1,     1,     0, ...,    62,     0,     0],
       ...,
       [66881,     4,     2, ...,   181,   103,   105],
       [66881,     4,     2, ...,   181,   103,   105],
       [66881,     4,     2, ...,   181,   103,   105]])

In [224]:
def build_feature_index(sparse_columns, varlen_sparse_columns):

    features = OrderedDict()
    start = 0

    # 依次对稀疏数据与变长数据进行处理
    for feat in sparse_columns:
        features[feat] = (start, start + 1)
        start += 1
    for feat in varlen_sparse_columns:
        features[feat] = (start, start + VARLEN_SPARSE_MAXLEN)
        start += VARLEN_SPARSE_MAXLEN

    return features

target_sparse_feature_names = ['user_id', 'age', 'gender', 'residence', 'city',
                               'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name','device_size',
                               'net_type', 'task_id', 'adv_id', 'creat_type_cd',
                               'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
                               'hispace_app_tags', 'app_second_class', 'app_score',
                               'pt_d', 'u_refreshTimes', 'u_feedLifeCycle']
target_varlen_sparse_feature_names = ['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
                                      'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
                                      'u_newsCatInterestsST']

target_feature_index = build_feature_index(target_sparse_feature_names, target_varlen_sparse_feature_names)

## 实现目标域的训练过程与测试过程

In [225]:
target_feature_index

OrderedDict([('user_id', (0, 1)),
             ('age', (1, 2)),
             ('gender', (2, 3)),
             ('residence', (3, 4)),
             ('city', (4, 5)),
             ('city_rank', (5, 6)),
             ('series_dev', (6, 7)),
             ('series_group', (7, 8)),
             ('emui_dev', (8, 9)),
             ('device_name', (9, 10)),
             ('device_size', (10, 11)),
             ('net_type', (11, 12)),
             ('task_id', (12, 13)),
             ('adv_id', (13, 14)),
             ('creat_type_cd', (14, 15)),
             ('adv_prim_id', (15, 16)),
             ('inter_type_cd', (16, 17)),
             ('slot_id', (17, 18)),
             ('site_id', (18, 19)),
             ('spread_app_id', (19, 20)),
             ('hispace_app_tags', (20, 21)),
             ('app_second_class', (21, 22)),
             ('app_score', (22, 23)),
             ('pt_d', (23, 24)),
             ('u_refreshTimes', (24, 25)),
             ('u_feedLifeCycle', (25, 26)),
             ('a

In [227]:
X.shape

(8651575, 61)

In [336]:
user_info_feature_names = ['user_id', 'age', 'gender', 'residence', 'city', 'city_rank',
                           'series_dev', 'series_group', 'emui_dev', 'device_name', 'device_size']
target_ad_feature_names = ['net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd',
                           'slot_id', 'site_id', 'spread_app_id', 'hispace_app_tags', 'app_second_class', 'app_score']
interact_ad_feature_names = ['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
                             'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003']
content_info_feature_names = ['pt_d', 'u_newsCatInterestsST', 'u_refreshTimes',  'u_feedLifeCycle']

In [230]:
embedding_dict

ModuleDict(
  (log_id): Embedding(1176633, 10)
  (label): Embedding(2, 10)
  (user_id): Embedding(66882, 10)
  (age): Embedding(8, 10)
  (gender): Embedding(3, 10)
  (residence): Embedding(36, 10)
  (city): Embedding(341, 10)
  (city_rank): Embedding(4, 10)
  (series_dev): Embedding(27, 10)
  (series_group): Embedding(7, 10)
  (emui_dev): Embedding(27, 10)
  (device_name): Embedding(258, 10)
  (device_size): Embedding(1581, 10)
  (net_type): Embedding(6, 10)
  (task_id): Embedding(11984, 10)
  (adv_id): Embedding(13579, 10)
  (creat_type_cd): Embedding(9, 10)
  (adv_prim_id): Embedding(565, 10)
  (inter_type_cd): Embedding(4, 10)
  (slot_id): Embedding(61, 10)
  (site_id): Embedding(1, 10)
  (spread_app_id): Embedding(116, 10)
  (hispace_app_tags): Embedding(43, 10)
  (app_second_class): Embedding(20, 10)
  (app_score): Embedding(3, 10)
  (pt_d): Embedding(6156, 10)
  (u_refreshTimes): Embedding(10, 10)
  (u_feedLifeCycle): Embedding(8, 10)
  (ad_click_list_v001): Embedding(20922, 10)


In [232]:
X = torch.from_numpy(X)

In [236]:
X.shape

torch.Size([8651575, 61])

In [270]:
target_sparse_embedding_list = OrderedDict()

for feat in target_sparse_feature_names + target_varlen_sparse_feature_names:
    target_sparse_embedding_list[feat] = embedding_dict[feat](
        X[:, target_feature_index[feat][0]:target_feature_index[feat][1]].long()
    )

In [287]:
target_sparse_embedding_list

OrderedDict([('user_id',
              tensor([[[-0.9602, -0.9755,  0.5034,  ..., -1.4086,  0.2106,  0.8161]],
              
                      [[-0.9602, -0.9755,  0.5034,  ..., -1.4086,  0.2106,  0.8161]],
              
                      [[-0.9602, -0.9755,  0.5034,  ..., -1.4086,  0.2106,  0.8161]],
              
                      ...,
              
                      [[ 0.9637, -0.0375,  0.8596,  ..., -0.8585,  1.4224,  0.4617]],
              
                      [[ 0.9637, -0.0375,  0.8596,  ..., -0.8585,  1.4224,  0.4617]],
              
                      [[ 0.9637, -0.0375,  0.8596,  ..., -0.8585,  1.4224,  0.4617]]],
                     grad_fn=<EmbeddingBackward0>)),
             ('age',
              tensor([[[-1.4797,  0.1056,  1.8432,  ..., -0.7230, -1.7041,  3.1675]],
              
                      [[-1.4797,  0.1056,  1.8432,  ..., -0.7230, -1.7041,  3.1675]],
              
                      [[-1.4797,  0.1056,  1.8432,  ..., -0.7230,

In [337]:
b = len(target_sparse_embedding_list['user_id'])
b

8651575

In [345]:
user_info_part = [target_sparse_embedding_list[feat].reshape(batch_size, -1) for feat in user_info_feature_names]
target_ad_part = [target_sparse_embedding_list[feat].reshape(batch_size, -1) for feat in target_ad_feature_names]
interact_ad_part = [(target_sparse_embedding_list[feat] * mask_dick[feat]).reshape(batch_size, -1) for feat in interact_ad_feature_names]
content_info_part = []
for feat in content_info_feature_names:
    if feat == 'u_newsCatInterestsST':
        content_info_part.append((target_sparse_embedding_list[feat] * mask_dick[feat]).reshape(batch_size, -1))
    else:
        content_info_part.append(target_sparse_embedding_list[feat].reshape(batch_size, -1))

In [346]:
batch_size = b
user_info_part = torch.cat(user_info_part, dim=-1)
target_ad_part = torch.cat(target_ad_part, dim=-1)
interact_ad_part = torch.cat(interact_ad_part, dim=-1)
content_info_part = torch.cat(content_info_part, dim=-1)
target_sparse_part_list = [user_info_part, target_ad_part, interact_ad_part, content_info_part]

In [347]:
user_info_part.shape

torch.Size([8651575, 110])

In [348]:
target_ad_part.shape, interact_ad_part.shape, content_info_part.shape

(torch.Size([8651575, 120]),
 torch.Size([8651575, 300]),
 torch.Size([8651575, 80]))

In [371]:
len(target_sparse_feature_names), len(target_varlen_sparse_feature_colnums)

(26, 7)

In [372]:
len(target_sparse_feature_names) * 10 + len(target_varlen_sparse_feature_colnums) * 5 * 10

610

## 注意力机制编写

In [349]:
class Interest_Level_Attention(nn.Module):

    def __init__(self):
        super().__init__()

        self.interset_attention_weight = nn.Parameter(torch.ones(4))
        self.init_weight()

    def forward(self, X):
        out = [self.interset_attention_weight[i] * X_ for i, X_ in enumerate(X)]
        out = torch.cat(out, dim=-1)
        return out

    def init_weight(self, init_std=0.0001):
        torch.nn.init.normal_(self.interset_attention_weight, mean=0, std=init_std)

In [350]:
attention = Interest_Level_Attention()

In [449]:
colunms = 610
b = 2000
p1 = torch.randn([b, 110], requires_grad=True)
p2 = torch.randn([b, 120], requires_grad=True)
p3 = torch.randn([b, 300], requires_grad=True)
p4 = torch.randn([b, 80], requires_grad=True)
data = [p1,p2,p3,p4]
cat_data = torch.cat(data, dim=-1)

In [388]:
import torch.nn.functional as F

In [437]:
vu = torch.mm(gu, F.relu(torch.mm(cat_data, Vu) + bu)).exp()

In [454]:
colunms_dim_list = [110, 120, 300, 80]
part_nums = len(colunms_dim_list)
all_colunms_dim = np.array(colunms_dim_list).sum()
b = 2000
V_params = nn.ParameterList([nn.Parameter(torch.Tensor(all_colunms_dim, 1)) for i in range(part_nums)])
g_params = nn.ParameterList([nn.Parameter(torch.Tensor(1, b)) for i in range(part_nums)])
b_params =nn.ParameterList([nn.Parameter(torch.Tensor(1)) for i in range(part_nums)])

In [457]:
init_std = 0.0001
for weight in V_params:
    torch.nn.init.normal_(weight, mean=0, std=init_std)
for weight in g_params:
    torch.nn.init.normal_(weight, mean=0, std=init_std)
for weight in b_params:
    torch.nn.init.normal_(weight, mean=0, std=init_std)

In [463]:
weight = [(torch.mm(g_params[i], F.relu(torch.mm(cat_data, V_params[i]))) + b_params[i]).exp()  for i in range(part_nums)]

In [464]:
weight

[tensor([[1.0000]], grad_fn=<ExpBackward0>),
 tensor([[1.0000]], grad_fn=<ExpBackward0>),
 tensor([[1.0002]], grad_fn=<ExpBackward0>),
 tensor([[1.0000]], grad_fn=<ExpBackward0>)]

In [466]:
out = torch.cat([weight[i] * data[i] for i in range(part_nums)], dim=-1)

In [467]:
out.shape

torch.Size([2000, 610])

## 模型测试

In [4]:
from model import CrossDomainNet

batch_size = 4092
model = CrossDomainNet(batch_size=batch_size)

In [6]:
model.target_mask_dict['ad_click_list_v001'].shape

torch.Size([8651575, 5, 10])

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

from model import CrossDomainNet
from utils.feature_deal import target_feature_process, target_sparse_feature_names, target_varlen_sparse_feature_names

In [11]:
epoch_size = 2
batch_size = 4092
SEED = 1024
metrics = ['binary_crossentropy', 'auc']
device = torch.device('cpu')
    
data = pd.read_csv("ctr_data/train/train_data_ads.csv")
# 获取标签数据
y = data['label'].values
# 获取训练数据（这里需要先编码 -> 再拼接，形成一个矩阵）
values_dick = target_feature_process(data, target_sparse_feature_names, target_varlen_sparse_feature_names,
                                     model.target_labelencoder_dict)
x = [values_dick[feat] for feat in target_sparse_feature_names + target_varlen_sparse_feature_names]
# 需要对数据进行扩维才能进行矩阵拼接
for i in range(len(x)):
    if len(x[i].shape) == 1:
        x[i] = np.expand_dims(x[i], axis=1)
# 对列表x进行拼接
X = np.concatenate(x, axis=-1)
print('X:', X.shape)

# 数据切分处理
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

X: (7675517, 61)


In [25]:
feat = 'ad_click_list_v003'
X[:, model.target_feature_index[feat][0]:model.target_feature_index[feat][1]]

array([[ 12, 109, 197,   0,   0],
       [ 12, 109, 197,   0,   0],
       [ 12, 109, 197,   0,   0],
       ...,
       [ 82, 100,   0,   0,   0],
       [ 82, 100,   0,   0,   0],
       [ 82, 100,   0,   0,   0]])

In [27]:
# 对训练数据构建dataloader
train_tensor_data = Data.TensorDataset(
    torch.from_numpy(X_train),
    torch.from_numpy(y_train))
train_loader = DataLoader(dataset=train_tensor_data, shuffle=True, batch_size=batch_size)

In [40]:
sample_x, sample_y = iter(train_loader).next()

In [41]:
sample_x.shape, sample_y.shape

(torch.Size([4092, 61]), torch.Size([4092]))

In [42]:
model.target_feature_index

OrderedDict([('user_id', (0, 1)),
             ('age', (1, 2)),
             ('gender', (2, 3)),
             ('residence', (3, 4)),
             ('city', (4, 5)),
             ('city_rank', (5, 6)),
             ('series_dev', (6, 7)),
             ('series_group', (7, 8)),
             ('emui_dev', (8, 9)),
             ('device_name', (9, 10)),
             ('device_size', (10, 11)),
             ('net_type', (11, 12)),
             ('task_id', (12, 13)),
             ('adv_id', (13, 14)),
             ('creat_type_cd', (14, 15)),
             ('adv_prim_id', (15, 16)),
             ('inter_type_cd', (16, 17)),
             ('slot_id', (17, 18)),
             ('site_id', (18, 19)),
             ('spread_app_id', (19, 20)),
             ('hispace_app_tags', (20, 21)),
             ('app_second_class', (21, 22)),
             ('app_score', (22, 23)),
             ('pt_d', (23, 24)),
             ('u_refreshTimes', (24, 25)),
             ('u_feedLifeCycle', (25, 26)),
             ('a

In [43]:
for feat in model.target_varlen_sparse_colunms:
    print(feat)

ad_click_list_v001
ad_click_list_v002
ad_click_list_v003
ad_close_list_v001
ad_close_list_v002
ad_close_list_v003
u_newsCatInterestsST


In [75]:
feat = 'ad_click_list_v001'
feat_value = sample_x[:, model.target_feature_index[feat][0]:model.target_feature_index[feat][1]]
seq_length = feat_value.count_nonzero(dim=1)  # 统计每一行的非0个数

In [51]:
feat_value

tensor([[30,  0,  0,  0,  0],
        [30,  0,  0,  0,  0],
        [30,  0,  0,  0,  0],
        ...,
        [30,  0,  0,  0,  0],
        [30,  0,  0,  0,  0],
        [30,  0,  0,  0,  0]])

In [81]:
seq_length = seq_length.unsqueeze(-1)

In [92]:
seq_length.max()

tensor(5)

In [74]:
def _sequence_mask(lengths, maxlen=None, dtype=torch.bool):
    # Returns a mask tensor representing the first N positions of each cell.
    if maxlen is None:
        maxlen = lengths.max()
    row_vector = torch.arange(0, maxlen, 1).to(lengths.device)
    matrix = torch.unsqueeze(lengths, dim=-1)
    mask = row_vector < matrix
    mask.type(dtype)
    return mask

In [84]:
mask = _sequence_mask(seq_length)
mask.shape

torch.Size([4092, 1, 5])

In [87]:
from collections import OrderedDict

def get_target_mask_dick(x):
    target_mask_dick = OrderedDict()
    for feat in model.target_varlen_sparse_colunms:
        # 根据feature_index找到相关的变长特征编码值
        feat_value = x[:, model.target_feature_index[feat][0]:model.target_feature_index[feat][1]]
        seq_length = feat_value.count_nonzero(dim=1)  # 统计每一行的非0个数
        seq_length = seq_length.unsqueeze(-1)         # (batch_size, ) -> (batch_size, 1)
        # 获取掩码后，需要拓展到embedding_dim的维度
        mask = _sequence_mask(seq_length)
        # (batch_size, 1, maxlen) -> (batch_size, maxlen, 1) -> batch_size, maxlen, embedding_dim)
        mask = torch.transpose(mask, 1, 2)
        embedding_size = model.target_embedding_dict[feat].embedding_dim
        mask = torch.repeat_interleave(mask, embedding_size, dim=2)
        target_mask_dick[feat] = mask
        
    return target_mask_dick

In [ ]:
target_mask_dick = get_target_mask_dick(sample_x)

## 评价方式

In [93]:
from train import train_one_epoch, eval_one_epoch, get_metrics

In [94]:
metrics = get_metrics(['binary_crossentropy', 'auc'])

In [95]:
metrics

{'binary_crossentropy': <function sklearn.metrics._classification.log_loss(y_true, y_pred, *, eps=1e-15, normalize=True, sample_weight=None, labels=None)>,
 'auc': <function sklearn.metrics._ranking.roc_auc_score(y_true, y_score, *, average='macro', sample_weight=None, max_fpr=None, multi_class='raise', labels=None)>}

In [ ]:
train_result = {}

In [108]:
for name, metric_fun in metrics.items():
    if name not in train_result:
        train_result[name] = []
    train_result[name].append(metric_fun(
        sample_y.cpu().data.numpy(), pred_y.cpu().data.numpy().astype("float64")))

In [110]:
epoch_logs = {}

In [111]:
for name, result in train_result.items():
    epoch_logs[name] = np.sum(result) / 10

In [112]:
epoch_logs

{'binary_crossentropy': 0.29495023517294067, 'auc': 0.14743239890845944}

In [119]:
train_info = str()
for name in epoch_logs.keys():
    train_info += "{}: {}\n".format(name, epoch_logs[name])
print(train_info)

binary_crossentropy: 0.29495023517294067
auc: 0.14743239890845944



## 测试脚本

In [158]:
padding_nums = batch_size - len(X) % batch_size
padding_colunms = X.shape[-1]

In [159]:
padding_data = np.zeros([padding_nums, padding_colunms])
test_X = np.concatenate([X, padding_data])

In [168]:
padding_nums, test_X.shape, X.shape

(387, (7675904, 61), (7675517, 61))

In [169]:
len(test_X) % batch_size

0

In [170]:
y_pred = test_X[:,0]

In [173]:
y_pred[:-padding_nums].shape

(7675517,)

In [175]:
y_pred[:len(y_pred)-padding_nums].shape

(7675517,)

In [176]:
data = pd.read_csv("ctr_data/test/test_data_ads.csv")

In [179]:
model = CrossDomainNet()
model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [181]:
X.shape

(7675517, 61)

- 解决方法2：拿前面的数据重新填补到后面的数据中

In [188]:
padding_data = X[:padding_nums, :]

In [189]:
padding_data.shape

(387, 61)

In [191]:
test_X = np.concatenate([X, padding_data])
len(test_X) % batch_size

0

In [195]:
data

,log_id,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,242295,100002,2,3,13,225,2,31,3,21,...,21791^33265,2020^1236,352,24107,1218,173,202206101201,86^15^86^112^191,0,15
1,242300,100002,2,3,13,225,2,31,3,21,...,21791^33265,2020^1236,352,24107,1218,173,202206101154,86^15^86^112^191,0,15
2,242299,100002,2,3,13,225,2,31,3,21,...,21791^33265,2020^1236,352,24107,1218,173,202206101228,86^15^86^112^191,0,15
3,242296,100002,2,3,13,225,2,31,3,21,...,21791^33265,2020^1236,352,24107,1218,173,202206101144,86^15^86^112^191,0,15
4,242293,100002,2,3,13,225,2,31,3,21,...,21791^33265,2020^1236,352,24107,1218,173,202206101145,86^15^86^112^191,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976053,653084,287180,6,4,33,319,3,27,2,11,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206100920,65^192^71^199^20,7,17
976054,653074,287180,6,4,33,319,3,27,2,11,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206101155,65^192^71^199^20,7,17
976055,653076,287180,6,4,33,319,3,27,2,11,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206101157,65^192^71^199^20,7,17
976056,653077,287180,6,4,33,319,3,27,2,11,...,34890^21489^34426^22929^34057,1432^1858^1717^1156^2056,190^208,24107,1218,173,202206101158,65^192^71^199^20,7,17


# 引入源域

In [18]:
# 导入源域的训练集与训练集
now = time.time()
source_train = pd.read_csv('ctr_data/train/train_data_feeds.csv')
source_test = pd.read_csv('ctr_data/test/test_data_feeds.csv')
# source_all_data = pd.concat([source_train, source_test])
print('load time: {} s'.format(time.time() - now))

load time: 13.29396104812622 s


首先要对源域数据进行筛选处理，剔除掉目标域没有出现过的用户ID的数据

In [19]:
# 导入目标域的训练集与训练集
target_train = pd.read_csv('ctr_data/train/train_data_ads.csv')
target_test = pd.read_csv('ctr_data/test/test_data_ads.csv')
target_all_data = pd.concat([target_train, target_test])

- 首先用目标域的训练集对筛选后的源域数据集进行编码

In [58]:
# 筛选源域训练集数据
target_train_userid = target_train['user_id'].unique()
source_train = source_train[source_train['u_userId'].isin(target_train_userid)]

# 筛选源域测试集数据
target_test_userid = target_test['user_id'].unique()
source_test = source_test[source_test['u_userId'].isin(target_test_userid)]

# 将源域训练集与测试集数据合并起来编码
source_all_data = pd.concat([source_train, source_test])

In [25]:
source_train.to_csv('ctr_data/train/train_data_feeds_drop.csv', index=False)
source_test.to_csv('ctr_data/test/test_data_feeds_drop.csv', index=False)

In [27]:
len(source_train), len(source_test)

(2475843, 286167)

- 对筛选后的源域数据进行编码

In [88]:
from utils.feature_deal import get_embedding_mask_lbe_dict, build_feature_index, \
    source_sparse_feature_names, source_varlen_sparse_feature_names, target_sparse_feature_names, target_varlen_sparse_feature_names

In [23]:
target_embedding_dict, target_labelencoder_dict = \
        get_embedding_mask_lbe_dict(target_all_data, target_sparse_feature_names, target_varlen_sparse_feature_names)

- 判断几个特征列表中是否包含^分隔符

In [71]:
feat = 'i_entities'
feat_data = source_all_data[feat].values

In [83]:
# 确实值填充
source_all_data[feat][source_all_data[feat].isna()] = 0

/tmp/ipykernel_69890/4134956217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_all_data[feat][source_all_data[feat].isna()] = 0


最后发现'i_entities'特征中含有确实值，导致了编码错误

In [86]:
source_all_data.nunique()

u_userId                 66882
u_phonePrice                 7
u_browserLifeCycle           8
u_browserMode                8
u_feedLifeCycle              8
u_refreshTimes              10
u_newsCatInterests      145465
u_newsCatDislike           695
u_newsCatInterestsST    582181
u_click_ca2_news        679260
i_docId                 105580
i_s_sourceId              4010
i_regionEntity             376
i_cat                      209
i_entities               99416
i_dislikeTimes              10
i_upTimes                   10
i_dtype                      5
e_ch                        19
e_m                        258
e_po                        27
e_pl                      3022
e_rn                        99
e_section                    2
e_et                      4136
label                        2
cillabel                     2
pro                         89
dtype: int64

## 源域特征编码与索引

In [4]:
from utils.feature_deal import get_embedding_mask_lbe_dict, build_feature_index, \
    target_varlen_sparse_feature_names, target_sparse_feature_names, \
    user_info_feature_names, target_ad_feature_names, interact_ad_feature_names, content_info_feature_names, \
    source_sparse_feature_names, source_varlen_sparse_feature_names, \
    SPARSE_EMBEDDING_DIM, VARLEN_SPARSE_EMBEDDING_DIM, VARLEN_SPARSE_MAXLEN

In [2]:
source_train = pd.read_csv('ctr_data/train/train_data_feeds_drop.csv')
source_test = pd.read_csv('ctr_data/test/test_data_feeds_drop.csv')
source_all_data = pd.concat([source_train, source_test])

In [7]:
source_embedding_dict, source_labelencoder_dick = \
            get_embedding_mask_lbe_dict(source_all_data, source_sparse_feature_names,
                                        source_varlen_sparse_feature_names)

In [6]:
target_train = pd.read_csv("ctr_data/train/train_data_ads.csv")
target_test = pd.read_csv('ctr_data/test/test_data_ads.csv')
target_all_data = pd.concat([target_train, target_test])

In [7]:
target_embedding_dict, target_labelencoder_dick = \
            get_embedding_mask_lbe_dict(target_all_data, target_sparse_feature_names,
                                        target_varlen_sparse_feature_names)

In [61]:
source_feature_index = build_feature_index(source_sparse_feature_names, source_varlen_sparse_feature_names)

In [62]:
source_feature_index

OrderedDict([('u_userId', (0, 1)),
             ('u_phonePrice', (1, 2)),
             ('u_browserLifeCycle', (2, 3)),
             ('u_browserMode', (3, 4)),
             ('u_feedLifeCycle', (4, 5)),
             ('u_refreshTimes', (5, 6)),
             ('i_docId', (6, 7)),
             ('i_s_sourceId', (7, 8)),
             ('i_regionEntity', (8, 9)),
             ('i_cat', (9, 10)),
             ('i_dislikeTimes', (10, 11)),
             ('i_upTimes', (11, 12)),
             ('i_dtype', (12, 13)),
             ('e_ch', (13, 14)),
             ('e_m', (14, 15)),
             ('e_po', (15, 16)),
             ('e_pl', (16, 17)),
             ('e_rn', (17, 18)),
             ('e_section', (18, 19)),
             ('e_et', (19, 20)),
             ('u_newsCatInterests', (20, 25)),
             ('u_newsCatDislike', (25, 30)),
             ('u_newsCatInterestsST', (30, 35)),
             ('u_click_ca2_news', (35, 40)),
             ('i_entities', (40, 45))])

## 对源域数据进行编码

In [2]:
source_train = pd.read_csv('ctr_data/train/train_data_feeds_drop.csv')
source_test = pd.read_csv('ctr_data/test/test_data_feeds_drop.csv')
source_all_data = pd.concat([source_train, source_test])

In [5]:
source_embedding_dict, source_labelencoder_dick = \
            get_embedding_mask_lbe_dict(source_all_data, source_sparse_feature_names,
                                        source_varlen_sparse_feature_names)

In [12]:
source_labelencoder_dick['u_userId'] = target_labelencoder_dick['user_id']

In [24]:
# 对源域数据进行处理
def source_feature_process(data, sparse_columns, varlen_sparse_columns, labelencoder_dict):

    values_dick = OrderedDict()

    # 为列表数据构建子特征的dataframe
    def get_varlen_sparse_feature_dataframe(dataframe, feature):
        varlen_sparse_feature_data = dataframe[feature].values  # 获取列表数据
        varlen_sparse_feature_datalists = [values_list.split('^') for values_list in varlen_sparse_feature_data]
        # 切分后的数据构建成dataframe形式，并用0来替换填充为None的数值
        varlen_sparse_feature_dataframe = pd.DataFrame(data=varlen_sparse_feature_datalists).replace([None], ['0'])
        return varlen_sparse_feature_dataframe

    # 存储稀疏特征值
    for feat in sparse_columns:
        values_dick[feat] = labelencoder_dict[feat].fit_transform(data[feat])

    # 存储变长稀疏特征值
    for feat in varlen_sparse_columns:
        feat_dataframe = get_varlen_sparse_feature_dataframe(data, feat)
        feat_encoder_values = labelencoder_dict[feat].fit_transform(feat_dataframe.values.flatten())
        # 多值变长特征重新构建矩阵
        values_dick[feat] = np.array(feat_encoder_values).reshape(-1, 5)

    # 源域数据中含有多个监督信息，这里用于将其存储在字典中
    supervise_features = ['label', 'cillabel', 'pro']
    for feat in supervise_features:
        values_dick[feat] = data[feat].values

    return values_dick

In [25]:
source_train_values_dick = source_feature_process(source_train, source_sparse_feature_names,
                                        source_varlen_sparse_feature_names, source_labelencoder_dick)

In [297]:
source_train_values_dick

OrderedDict([('u_userId',
              array([ 5919,  5919,  5919, ..., 60300, 57705, 57705])),
             ('u_phonePrice', array([1, 1, 1, ..., 1, 4, 4])),
             ('u_browserLifeCycle', array([7, 7, 7, ..., 7, 7, 7])),
             ('u_browserMode', array([4, 4, 4, ..., 5, 5, 5])),
             ('u_feedLifeCycle', array([7, 7, 7, ..., 7, 7, 7])),
             ('u_refreshTimes', array([7, 7, 7, ..., 5, 7, 7])),
             ('i_docId',
              array([82755, 38151, 29501, ..., 82025, 79115, 38183])),
             ('i_s_sourceId',
              array([2964, 1842, 1842, ...,   90, 1449, 2498])),
             ('i_regionEntity', array([  0,   0, 201, ...,   0,   0,   0])),
             ('i_cat', array([ 16,  16, 110, ...,  16,  95, 106])),
             ('i_dislikeTimes', array([9, 0, 0, ..., 1, 0, 0])),
             ('i_upTimes', array([9, 0, 0, ..., 9, 9, 9])),
             ('i_dtype', array([3, 3, 3, ..., 0, 0, 1])),
             ('e_ch', array([17, 17, 17, ..., 17, 17, 17]

In [305]:
supervise_features = ['label', 'cillabel', 'pro']
source_train_X = [source_train_values_dick[feat] for feat in source_sparse_feature_names + source_varlen_sparse_feature_names + supervise_features]

In [306]:
def expand2cat(x):
    for i in range(len(x)):
        if len(x[i].shape) == 1:
            x[i] = np.expand_dims(x[i], axis=1)
    X = np.concatenate(x, axis=-1)
    return X

In [307]:
source_train_X = expand2cat(source_train_X)

In [308]:
source_train_X = pd.DataFrame(data=source_train_X)
source_train_X

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,5919,1,7,4,7,7,82755,2964,0,16,...,50,214,158964,0,0,0,0,-1,-1,0
1,5919,1,7,4,7,7,38151,1842,0,16,...,214,131,192061,18831,0,0,0,-1,-1,0
2,5919,1,7,4,7,7,29501,1842,201,110,...,203,77,107090,231535,292501,68092,220901,-1,-1,0
3,5919,1,7,4,7,7,3233,33,0,86,...,203,77,15315,85377,76443,348098,57819,-1,-1,0
4,5919,1,7,4,7,7,14856,3212,0,110,...,214,131,78346,39757,55304,103905,179090,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475838,60300,1,7,5,7,5,38903,3232,122,16,...,0,52,295915,118513,27983,353438,4575,-1,-1,0
2475839,60300,1,7,5,7,5,18264,1300,0,106,...,194,77,281637,306536,360269,209931,162422,1,-1,100
2475840,60300,1,7,5,7,5,82025,90,0,16,...,194,77,154503,97275,169228,258258,223190,-1,-1,0
2475841,57705,4,7,5,7,7,79115,1449,0,95,...,181,172,95672,24425,229088,157687,269361,-1,-1,0


## 根据id获取对于的索引矩阵

In [78]:
batch_size = 4096
target_train_sample = target_train.values[:batch_size, 2:]
target_train_sample[:, 0] = target_labelencoder_dick['user_id'].fit_transform(target_train_sample[:, 0])
target_train_sampleID = target_labelencoder_dick['user_id'].fit_transform(target_train_sample[:, 0])

In [79]:
target_train_sample.shape, target_train_sample

((4096, 33),
 array([[0, 3, 2, ..., '39^220^16', 0, 15],
        [0, 3, 2, ..., '39^220^16', 0, 15],
        [0, 3, 2, ..., '39^220^16', 0, 15],
        ...,
        [123, 3, 2, ..., '65^65^183^2', 4, 17],
        [123, 3, 2, ..., '65^65^183^2', 4, 17],
        [123, 3, 2, ..., '65^65^183^2', 4, 17]], dtype=object))

In [64]:
unique_sampleID = np.unique(target_train_sampleID)

In [67]:
unique_sampleID

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123])

现在获得了训练过程中的一批ID号，先需要根据源域数据中，获取对于的ID数据source_train_X

In [59]:
source_train_X = pd.DataFrame(data=source_train_X)

In [309]:
source_train_sample = source_train_X[source_train_X[0].isin(unique_sampleID)]
source_train_sample

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
4162,37,5,6,6,6,0,16421,2357,0,206,...,0,0,92963,137725,311487,161118,259117,-1,-1,0
4163,37,5,6,6,6,0,87441,188,321,106,...,0,0,144538,113448,179290,284678,291769,-1,-1,0
12040,57,0,7,4,7,7,7128,188,371,166,...,132,131,116806,347265,281722,357274,3209,-1,-1,0
12041,57,0,7,4,7,7,12126,487,169,106,...,132,131,370188,268120,20309,301765,294839,-1,-1,0
12042,57,0,7,4,7,7,11417,1842,0,206,...,132,131,230810,276605,170905,132848,0,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470826,79,6,7,5,7,7,69257,754,0,160,...,75,174,111923,253035,217645,102971,341245,-1,-1,0
2470827,79,6,7,5,7,7,86594,2723,0,160,...,75,174,122212,138009,203716,139281,196869,-1,-1,0
2470828,79,6,7,5,7,7,7233,461,0,160,...,9,75,262603,184091,291992,308347,16425,-1,-1,0
2470829,79,6,7,5,7,7,68500,69,0,96,...,9,75,370226,328505,54702,131044,12812,-1,-1,0


## 对索引矩阵依次处理进行embedding处理

In [95]:
def sequence_mask(lengths, maxlen=None, dtype=torch.bool):
    # Returns a mask tensor representing the first N positions of each cell.
    if maxlen is None:
        maxlen = lengths.max()
    row_vector = torch.arange(0, maxlen, 1).to(lengths.device)
    matrix = torch.unsqueeze(lengths, dim=-1)
    mask = row_vector < matrix
    mask.type(dtype)
    return mask

In [96]:
# 对source_train_sample进行embedding与mask处理
def get_source_mask_dick(X):
    mask_dict = OrderedDict()
    for feat in source_varlen_sparse_feature_names:
        # 根据feature_index找到相关的变长特征编码值
        feat_value = X[:, source_feature_index[feat][0]:source_feature_index[feat][1]]
        seq_length = feat_value.count_nonzero(dim=1)  # 统计每一行的非0个数
        seq_length = seq_length.unsqueeze(-1)  # (batch_size, ) -> (batch_size, 1)
        # 获取掩码后，需要拓展到embedding_dim的维度 (这里直接固定编码长度)
        mask = sequence_mask(seq_length, maxlen=5)
        # (batch_size, 1, maxlen) -> (batch_size, maxlen, 1) -> batch_size, maxlen, embedding_dim)
        mask = torch.transpose(mask, 1, 2)
        embedding_size = source_embedding_dict[feat].embedding_dim
        mask = torch.repeat_interleave(mask, embedding_size, dim=2)
        mask_dict[feat] = mask

    return mask_dict

In [97]:
source_mask_dict = get_source_mask_dick(torch.tensor(source_train_sample.values))

In [99]:
source_mask_dict['u_newsCatInterests'].shape

torch.Size([5019, 5, 10])

In [311]:
source_train_sample

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
4162,37,5,6,6,6,0,16421,2357,0,206,...,0,0,92963,137725,311487,161118,259117,-1,-1,0
4163,37,5,6,6,6,0,87441,188,321,106,...,0,0,144538,113448,179290,284678,291769,-1,-1,0
12040,57,0,7,4,7,7,7128,188,371,166,...,132,131,116806,347265,281722,357274,3209,-1,-1,0
12041,57,0,7,4,7,7,12126,487,169,106,...,132,131,370188,268120,20309,301765,294839,-1,-1,0
12042,57,0,7,4,7,7,11417,1842,0,206,...,132,131,230810,276605,170905,132848,0,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470826,79,6,7,5,7,7,69257,754,0,160,...,75,174,111923,253035,217645,102971,341245,-1,-1,0
2470827,79,6,7,5,7,7,86594,2723,0,160,...,75,174,122212,138009,203716,139281,196869,-1,-1,0
2470828,79,6,7,5,7,7,7233,461,0,160,...,9,75,262603,184091,291992,308347,16425,-1,-1,0
2470829,79,6,7,5,7,7,68500,69,0,96,...,9,75,370226,328505,54702,131044,12812,-1,-1,0


In [313]:
source_train_sample = torch.tensor(source_train_sample.values)

In [109]:
def get_sparse_embedding_list(X, sparse_colunms, varlen_sparse_colunms, embedding_dict, feature_index):
    sparse_embedding_list = OrderedDict()
    for feat in sparse_colunms + varlen_sparse_colunms:
        sparse_embedding_list[feat] = embedding_dict[feat](
            X[:, feature_index[feat][0]:feature_index[feat][1]].long()
        )
    return sparse_embedding_list

In [317]:
source_sparse_embedding_list = get_sparse_embedding_list(source_train_sample, 
                                        source_sparse_feature_names, source_varlen_sparse_feature_names, 
                                        embedding_dict=source_embedding_dict, feature_index=source_feature_index)

In [151]:
source_sparse_embedding_list['u_userId'].shape

torch.Size([5019, 1, 10])

In [152]:
for feat in source_varlen_sparse_feature_names:
    source_sparse_embedding_list[feat] = source_sparse_embedding_list[feat] * source_mask_dict[feat]

现在就获得了embedding后的采样数据

In [319]:
source_train_sample.shape, source_sparse_embedding_list['u_userId'].shape

(torch.Size([5019, 48]), torch.Size([5019, 1, 10]))

In [324]:
# 对其进行reshape后拼接在一起
for feat in source_sparse_feature_names + source_varlen_sparse_feature_names:
    source_sparse_embedding_list[feat] = source_sparse_embedding_list[feat].reshape(len(source_train_sample), -1)

In [325]:
# source_sparse_embedding_part = []
# source_sparse_embedding_part.append(torch.tensor(source_train_sample[:, 0:1], dtype=torch.float32, requires_grad=True))
source_sparse_embedding_part = [source_sparse_embedding_list[feat] for feat in source_sparse_feature_names + source_varlen_sparse_feature_names]

In [326]:
source_sparse_embedding_part[0].shape

torch.Size([5019, 10])

获取拼接好的数据

In [327]:
source_sparse_embedding_part = torch.cat(source_sparse_embedding_part, dim=-1)
source_sparse_embedding_part.shape

torch.Size([5019, 450])

## 根据id，拼接在相关id的目标数据后

In [177]:
source_sparse_embedding_part

tensor([[-2.0704e-04,  1.7899e-05, -7.8976e-06,  ...,  7.8105e-05,
          5.1677e-05,  1.2507e-04],
        [-2.0704e-04,  1.7899e-05, -7.8976e-06,  ...,  7.4475e-05,
         -5.9975e-05,  1.6671e-04],
        [-1.4883e-05,  1.0168e-04,  1.6734e-04,  ..., -1.2719e-04,
          2.0179e-05, -2.1695e-06],
        ...,
        [-1.0178e-04,  1.9271e-05,  4.6684e-06,  ...,  9.5800e-05,
          5.3576e-07,  2.5516e-05],
        [-1.0178e-04,  1.9271e-05,  4.6684e-06,  ..., -1.8705e-05,
         -6.7904e-05, -1.0060e-04],
        [-1.0178e-04,  1.9271e-05,  4.6684e-06,  ...,  1.0376e-04,
          5.0546e-06, -1.2299e-04]], grad_fn=<CatBackward0>)

In [172]:
target_train_sample

array([[0, 3, 2, ..., '39^220^16', 0, 15],
       [0, 3, 2, ..., '39^220^16', 0, 15],
       [0, 3, 2, ..., '39^220^16', 0, 15],
       ...,
       [123, 3, 2, ..., '65^65^183^2', 4, 17],
       [123, 3, 2, ..., '65^65^183^2', 4, 17],
       [123, 3, 2, ..., '65^65^183^2', 4, 17]], dtype=object)

In [178]:
source_train_sample

tensor([[    37,      5,      6,  ..., 311487, 161118, 259117],
        [    37,      5,      6,  ..., 179290, 284678, 291769],
        [    57,      0,      7,  ..., 281722, 357274,   3209],
        ...,
        [    79,      6,      7,  ..., 291992, 308347,  16425],
        [    79,      6,      7,  ...,  54702, 131044,  12812],
        [    79,      6,      7,  ..., 221978, 288110, 191974]])

In [201]:
index = torch.where(source_train_sample[:, 0] == 0)[0]
index

tensor([1087, 1941, 4828])

In [209]:
embedding_array = torch.index_select(source_sparse_embedding_part, dim=0, index=index)
embedding_pooling = embedding_array.mean(dim=0)
embedding_array.shape, embedding_pooling.shape

(torch.Size([3, 450]), torch.Size([450]))

- 根据唯一的ID构建一个字典

In [212]:
def get_source_index(data, id):
    id_index = torch.where(data[:, 0] == id)[0]
    return id_index

In [210]:
def get_fuse_embedding(embedding_array, index):
    embedding_array = torch.index_select(embedding_array, dim=0, index=index)
    embedding_pooling = embedding_array.mean(dim=0)
    return embedding_pooling

In [330]:
source_train_sample

tensor([[37,  5,  6,  ..., -1, -1,  0],
        [37,  5,  6,  ..., -1, -1,  0],
        [57,  0,  7,  ..., -1, -1,  0],
        ...,
        [79,  6,  7,  ..., -1, -1,  0],
        [79,  6,  7,  ..., -1, -1,  0],
        [79,  6,  7,  ..., -1, -1,  0]])

In [213]:
id_embedding_dict = OrderedDict()

for id in unique_sampleID:
    # 根据ID获取源域对应数据ID的全部索引
    id_index = get_source_index(source_train_sample, id)
    
    # 根据对于ID的全部数据，进行融合
    embedding_pooling = get_fuse_embedding(source_sparse_embedding_part, id_index)
    
    # 将相关信息存储在一个字典中
    id_embedding_dict[id] = embedding_pooling

In [215]:
id_embedding_dict[0].shape

torch.Size([450])

In [234]:
target_train_sample = target_train_sample.astype('float16')
target_train_sample = torch.from_numpy(target_train_sample)
target_train_sample.shape

torch.Size([4096, 10])

In [235]:
target_train_sample

tensor([[  0.,   3.,   2.,  ...,   6.,  35., 312.],
        [  0.,   3.,   2.,  ...,   6.,  35., 312.],
        [  0.,   3.,   2.,  ...,   6.,  35., 312.],
        ...,
        [123.,   3.,   2.,  ...,   5.,  11., 176.],
        [123.,   3.,   2.,  ...,   5.,  11., 176.],
        [123.,   3.,   2.,  ...,   5.,  11., 176.]], dtype=torch.float16)

In [239]:
id_embedding_dict[0].shape

torch.Size([450])

In [347]:
target_train_sample_list = [id_embedding_dict[int(row_data[0])].unsqueeze(0) for row_data in target_train_sample]
target_train_sample_part = torch.cat(target_train_sample_list, dim=0)

In [354]:
target_train_sample_part[:, 10:].shape

torch.Size([4096, 440])

In [355]:
np.array([110, 120, 300, 80, 440]).sum()

1050

## 添加源域损失

In [359]:
label = source_train_sample[:, -3]
cillabel = source_train_sample[:, -2]
pro = source_train_sample[:, -1]

In [362]:
source_sparse_embedding_part.shape

torch.Size([5019, 450])

In [ ]:
from deepctr_torch.layers import DNN

s_dnn = DNN(inputs_dim=450,
            hidden_units=(256, 128),
            activation=dnn_activation, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout, use_bn=dnn_use_bn,
            init_std=init_std, device=device)

# 辅助损失

In [459]:
class Source_Linear(nn.Module):

    def __init__(self, input_dim, out_dim, activation='tanh'):
        super().__init__()

#         assert activation in ['tanh', 'relu'], "No such activation choose, activation must be in ['tanh', 'relu']"

        self.linear = nn.Linear(input_dim, out_dim)
#         if activation == 'tanh':
#             self.act = nn.Tanh()
#         elif activation == 'relu':
#             self.act = nn.ReLU()

    def forward(self, x):
        out = self.linear(x)
        return out

In [460]:
from deepctr_torch.layers import FM, DNN

dnn_hidden_units=(256, 128)
l2_reg_linear=0.00001
l2_reg_embedding=0.00001
l2_reg_dnn=0
init_std=0.0001
dnn_dropout=0.6
dnn_use_bn=True
dnn_activation='relu'
device='cpu'
batch_size=4096

source_dnn = DNN(inputs_dim=450,
                  hidden_units=dnn_hidden_units,
                  activation=dnn_activation, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout, use_bn=dnn_use_bn,
                  init_std=init_std, device=device)
# source_label_clf = nn.Linear(128, 1)
source_label_clf = Source_Linear(input_dim=128, out_dim=1, activation='tanh')

out1 = source_dnn(source_sparse_embedding_part)
out2 = source_label_clf(out1)

In [455]:
out1.shape

torch.Size([5019, 128])

In [479]:
# 对源域数据进行处理
def source_feature_process(data, sparse_columns, varlen_sparse_columns, labelencoder_dict):

    values_dick = OrderedDict()

    # 为列表数据构建子特征的dataframe
    def get_varlen_sparse_feature_dataframe(dataframe, feature):
        varlen_sparse_feature_data = dataframe[feature].values  # 获取列表数据
        varlen_sparse_feature_datalists = [values_list.split('^') for values_list in varlen_sparse_feature_data]
        # 切分后的数据构建成dataframe形式，并用0来替换填充为None的数值
        varlen_sparse_feature_dataframe = pd.DataFrame(data=varlen_sparse_feature_datalists).replace([None], ['0'])
        return varlen_sparse_feature_dataframe

    # 存储稀疏特征值
    for feat in sparse_columns:
        values_dick[feat] = labelencoder_dict[feat].fit_transform(data[feat])

    # 存储变长稀疏特征值
    for feat in varlen_sparse_columns:
        feat_dataframe = get_varlen_sparse_feature_dataframe(data, feat)
        feat_encoder_values = labelencoder_dict[feat].fit_transform(feat_dataframe.values.flatten())
        # 多值变长特征重新构建矩阵
        values_dick[feat] = np.array(feat_encoder_values).reshape(-1, 5)

    # 源域数据中含有多个监督信息，这里用于将其存储在字典中
    supervise_features = ['label', 'cillabel', 'pro']
    for feat in supervise_features:
        values_dick[feat] = data[feat].values

    # 将其进行维度扩增然后拼接成一个array矩阵
    source_datalist = [values_dick[feat] for feat in sparse_columns + varlen_sparse_columns + supervise_features]
    for i in range(len(source_datalist)):
        if len(source_datalist[i].shape) == 1:
            source_datalist[i] = np.expand_dims(source_datalist[i], axis=1)
    source_datalist = np.concatenate(source_datalist, axis=-1)

    # 构建成dataframe以便与索引
    source_dataframe = pd.DataFrame(data=source_datalist)

    return source_dataframe

In [478]:
from utils.feature_deal import get_embedding_mask_lbe_dict, build_feature_index, \
    target_varlen_sparse_feature_names, target_sparse_feature_names, \
    user_info_feature_names, target_ad_feature_names, interact_ad_feature_names, content_info_feature_names, \
    source_sparse_feature_names, source_varlen_sparse_feature_names, \
    SPARSE_EMBEDDING_DIM, VARLEN_SPARSE_EMBEDDING_DIM, VARLEN_SPARSE_MAXLEN

In [480]:
from model import CrossDomainNet
from utils.feature_deal import target_feature_process, source_feature_process,  \
    target_sparse_feature_names, target_varlen_sparse_feature_names

batch_size = 4096
model = CrossDomainNet(batch_size=batch_size)

ImportError: cannot import name 'source_feature_process' from 'utils.feature_deal' (/home/fs/LLC/Competition/CTR/utils/feature_deal.py)